In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [24]:
class Net(nn.Module):
    
    def __init__(self):
        super().__init__() # python3 recommanded
        # Conv2d(input_channel,output_channel,square_size_filter,stride =1,padding = 0)
        self.conv1 = nn.Conv2d(1,6,4)
        self.conv2 = nn.Conv2d(6,16,4)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        print(x.size())
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        print(x.size())
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        print(x.size())
        x = x.view(-1, self.num_flat_features(x))# reshapce to fc
        print(x.size())
        x = F.relu(self.fc1(x))
        print(x.size())
        x = F.relu(self.fc2(x))
        print(x.size())
        x = self.fc3(x)
        print(x.size())
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features 
    
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [25]:
net.parameters

<bound method Module.parameters of Net(
  (conv1): Conv2d(1, 6, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

In [26]:
list(net.parameters())

[Parameter containing:
 tensor([[[[ 0.2243,  0.1254, -0.0322, -0.0711],
           [-0.1290,  0.1587,  0.2008,  0.1450],
           [ 0.2117, -0.2456,  0.2222, -0.0110],
           [-0.0588,  0.0218,  0.1535, -0.1991]]],
 
 
         [[[-0.1419,  0.1118,  0.2182,  0.1887],
           [-0.0693,  0.1367,  0.1914, -0.1554],
           [-0.1369, -0.1704, -0.2011, -0.0948],
           [ 0.1935,  0.0966, -0.1457, -0.1479]]],
 
 
         [[[ 0.2006, -0.0331, -0.1682, -0.1778],
           [-0.0963,  0.2295,  0.2192, -0.1679],
           [-0.1431,  0.0334, -0.2331, -0.0257],
           [ 0.1104,  0.2289,  0.0011,  0.0016]]],
 
 
         [[[ 0.0608, -0.1238,  0.0009,  0.0944],
           [-0.0030, -0.1937,  0.0211,  0.1545],
           [ 0.0401, -0.1840, -0.0582, -0.1611],
           [ 0.1690, -0.0349,  0.0256,  0.2177]]],
 
 
         [[[ 0.0170,  0.0590,  0.1460, -0.0055],
           [-0.1001, -0.0880, -0.0765,  0.0490],
           [ 0.0592,  0.2177, -0.1576,  0.0070],
           [-0.2006, -

In [27]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 4, 4])


In [28]:
input = torch.randn(1,1,32,32) # N,C,H,W
out = net(input)
print(out)

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])
torch.Size([1, 400])
torch.Size([1, 120])
torch.Size([1, 84])
torch.Size([1, 10])
tensor([[-0.0617, -0.1279,  0.0598, -0.0487,  0.0720,  0.0288,  0.0013, -0.0335,
          0.0453,  0.0583]], grad_fn=<AddmmBackward>)


In [29]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [30]:
# loss function
output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()
loss = criterion(output,target)
print(loss)

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])
torch.Size([1, 400])
torch.Size([1, 120])
torch.Size([1, 84])
torch.Size([1, 10])
tensor(0.4918, grad_fn=<MseLossBackward>)


In [31]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0140, -0.0118, -0.0082, -0.0068,  0.0020, -0.0092])


In [ ]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(),lr = 0.01)

optimizer.zero_grad() #make grad buffer to zero
output = net(input)
loss = criterion(output,target)
loss.backward()
optimizer.step()